In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

In [2]:
from master import * 

/usr/local/lib/python3.7/dist-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):
/usr/local/lib/python3.7/dist-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):


Log path = /root/src/aleister/log/logging.log


In [19]:
model_id = 'test004'

# args=['--symbol','BTC', '--execute_mode',"prepro",
#       '--model_id',model_id,'--model_name','slstm',
#       '--config_source','ini',
#       '--user',"Misaka00001", '--source','ipynb',
#       "--general_config_mode", "DOCKER","--private_api_mode","DEFAULT",
#       '-train_sd','20210101','-train_ed','20210103',
#       '-valid_sd','20210104','-valid_ed','20210105',
#       '-test_sd','20210106','-test_ed','20210107']

# args=['--symbol','BTC', '--execute_mode',"train",
#       '--model_id',model_id,'--model_name','slstm',
#       '--config_source','ini',
#       '--user',"Misaka00001", '--source','ipynb',
#       "--general_config_mode", "DOCKER","--private_api_mode","DEFAULT",
#       '-train_sd','20210101','-train_ed','20210103',
#       '-valid_sd','20210104','-valid_ed','20210105',
#       '-test_sd','20210106','-test_ed','20210107']

# args=['--symbol','BTC', '--execute_mode',"gtrain",
#       '--model_id',model_id,'--model_name','slstm',
#       '--config_source','ini',
#       '--user',"Misaka00001", '--source','ipynb',
#       "--general_config_mode", "DOCKER","--private_api_mode","DEFAULT",
#       '-train_sd','20210101','-train_ed','20210103',
#       '-valid_sd','20210104','-valid_ed','20210105',
#       '-test_sd','20210106','-test_ed','20210107']

# args=['--symbol','BTC', '--execute_mode',"deploy_model",
#       '--model_id',model_id,'--model_name','slstm',
#       '--config_source','ini',
#       '--user',"Misaka00001", '--source','ipynb',
#       "--general_config_mode", "DOCKER","--private_api_mode","DEFAULT",
#       '-train_sd','20210101','-train_ed','20210103',
#       '-valid_sd','20210104','-valid_ed','20210105',
#       '-test_sd','20210106','-test_ed','20210107']


args=['--symbol','BTC', '--execute_mode',"rpredict",
      '--model_id',model_id,'--model_name','slstm',
      '--config_source','ini',
      '--user',"Misaka00001", '--source','ipynb',
      "--general_config_mode", "DOCKER","--private_api_mode","DEFAULT",
      '-train_sd','20210101','-train_ed','20210103',
      '-valid_sd','20210104','-valid_ed','20210105',
      '-test_sd','20210106','-test_ed','20210107']

# args=['--symbol','BTC', '--execute_mode',"train",
#       '--model_id','test002','--model_name','cgm',
#       '--config_source','ini',
#       '--user',"Misaka00001", '--source','ipynb',
#       "--general_config_mode", "DOCKER","--private_api_mode","DEFAULT",
#       '-train_sd','20210101','-train_ed','20210103',
#       '-valid_sd','20210104','-valid_ed','20210105',
#       '-test_sd','20210106','-test_ed','20210107']

In [20]:
parser = make_parser()
arg_dict = vars(parser.parse_args(args))

# meta
_id = arg_dict["model_id"]
general_config_mode = arg_dict["general_config_mode"].upper()
private_api_mode = arg_dict["private_api_mode"].upper()
sym = arg_dict["symbol"]

model_name = arg_dict["model_name"].upper()

om = OperateMaster()
om.load_meta(_id, model_name, sym, general_config_mode, private_api_mode)
om.init_prepro()

# mlflow
mlflow_tags = {
    MLFLOW_USER: arg_dict["user"],
    MLFLOW_SOURCE_NAME: arg_dict["source"],
    MLFLOW_RUN_NAME: f"TRAIN_{dt.now().strftime('%y%m%d%H%M%s')}"
}
mlflow_client_kwargs = {
    "tracking_uri": os.environ['MLFLOW_TRACKING_URI']
}
om.set_mlflow_settings(mlflow_client_kwargs, mlflow_tags)

# load conofigs into each module
om.init_learning()


Log path = /root/src/aleister/log/logging.log
Log path = /root/src/aleister/log/logging.log


In [22]:

if arg_dict["execute_mode"] == "prepro":
    pass
    om.init_dataGen()
    # period
    train_start = arg_dict["train_start_date"] if "train_start_date" in arg_dict.keys() else None
    train_end = arg_dict["train_end_date"] if "train_end_date" in arg_dict.keys() else None
    valid_start = arg_dict["valid_start_date"] if "valid_start_date" in arg_dict.keys() else None
    valid_end = arg_dict["valid_end_date"] if "valid_end_date" in arg_dict.keys() else None
    test_start = arg_dict["test_start_date"] if "test_start_date" in arg_dict.keys() else None
    test_end = arg_dict["test_end_date"] if "test_end_date" in arg_dict.keys() else None
    # om.preprocessing(sym, train_start, train_end, valid_start, valid_end, test_start, test_end)
else:
    if arg_dict["execute_mode"] == "train":
        om.init_learning()
        # om.train()
    elif arg_dict["execute_mode"] == "gtrain":
        om.init_learning()
        # om.gtrain()
    elif arg_dict["execute_mode"] == "deploy_model":
        om.deploy_best_model()
    elif arg_dict["execute_mode"] == "rpredict":
        om.init_dataGen()
        # om.realtime_predict()


Log path = /root/src/aleister/log/logging.log
Log path = /root/src/aleister/log/logging.log


In [8]:
le = om.le
fp = om.fp
dg = om.dg

## Preprocessing

In [ ]:
om.preprocessing(sym, train_start, train_end, valid_start, valid_end, test_start, test_end)

In [8]:
fp._logger.info(f"{train_start}, {train_end}, {valid_start}, {valid_end}, {test_start}, {test_end}")
fetch_start = min([_date for _date in [train_start, train_end, valid_start, valid_end, test_start, test_end] if
                   _date is not None])
fetch_end = max([_date for _date in [train_start, train_end, valid_start, valid_end, test_start, test_end] if
                 _date is not None])
# dg.get_hist_data(ch="trade", sym="BTC", sd="20220101", ed="20220107")

#TODO: outside
fetch_start=train_start
fetch_end=test_end
output_data_structure =fp.model_config.get("OUTPUT_DATA_STRUCTURE")
input_data_structure = fp.model_config.get("INPUT_DATA_STRUCTURE")



In [9]:
output_data_structure

'flatten_v1'

In [10]:
data_tree =dg.get_Xy(mode='train',input_data_structure=input_data_structure,output_data_structure=output_data_structure,sym=sym, sd=fetch_start, ed=fetch_end)
fp._logger.info("[DONE] Get prepro raw datas. {0}~{1}".format(fetch_start, fetch_end))

In [11]:
# Split train/test/val ineach leaf
term_splited_data_tree = fp.split_child_by_period(data_tree,train_start, train_end, valid_start, valid_end, test_start, test_end)
fp._logger.info("[DONE] Split by term.")

# Make each period tree
train_datas = fp.select_child_by_period(term_splited_data_tree, 'train')
test_datas = fp.select_child_by_period(term_splited_data_tree, 'test')
val_datas = fp.select_child_by_period(term_splited_data_tree, 'valid')
fp._logger.info("[DONE] picked by term.")

# to ndarray objects
train_datas = fp.get_input_structures(input_data_structure)(train_datas)
test_datas = fp.get_input_structures(input_data_structure)(test_datas, scalers=train_datas['scaler'])
val_datas = fp.get_input_structures(input_data_structure)(val_datas, scalers=train_datas['scaler'])
fp._logger.info("[DONE] To numpy and numpy ")

#  Note: If other data except  for X is using. Fix X_train": X_trains, "X_val":X_vals, "X_test": X_tests
fp.save_numpy_datas(**{
    "X_train": train_datas['data']['X'], "X_val": test_datas['data']['X'], "X_test": val_datas['data']['X'],
    "y_train": train_datas['data']['y'], "y_val": test_datas['data']['y'], "y_test": val_datas['data']['y'],
    "X_scaler": train_datas['scaler'],"note":{'input_dim':train_datas['input_dim']}
})

## Train 

In [35]:
obj_keys = ["X_train", "X_val", "y_train", "y_val", "note"]
X_trains, X_vals, y_trains, y_vals, note = fp.load_numpy_datas(obj_keys)
om.train_worker(X_trains, X_vals, y_trains, y_vals,  note )


61


In [34]:
predictions

array([[9.99979973e-01, 5.17890703e-06, 1.49594198e-05],
       [9.99979854e-01, 5.17760282e-06, 1.49675225e-05],
       [9.99979854e-01, 5.16807131e-06, 1.50279675e-05],
       ...,
       [1.07656269e-05, 3.91482763e-06, 9.99985337e-01],
       [1.08758677e-05, 3.94127801e-06, 9.99985218e-01],
       [9.99778450e-01, 1.91520576e-05, 2.02424315e-04]], dtype=float32)

## Gtrain

In [15]:
om.gtrain()

0


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:188: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:188: RuntimeWarning:

invalid value encountered in double_scalars



ValueError: need at least one array to concatenate

In [22]:
len(param_grid)

12

## train worker 

In [17]:


X_trains, X_vals, y_train, y_val, note = X_trains, X_vals, y_trains, y_vals, note
# train set up
model_params = {_k: le.hparams[_k] for _k in le.hparams["structure_params"]}
model_params['input_dim'] = note['input_dim']

le.get_model_instance(le.model_name, model_params)

fp.get_dataset_fn(le.hparams["dataset"])
dataset_params = {_k: le.hparams[_k] for _k in le.hparams["dataset_params"]}
train_loader, val_loader, _, _ = fp.get_dataloader(le.hparams["dataset"], X_trains, y_train, X_vals,
                                                        y_val, None, None, **dataset_params)

61


In [18]:
lossfn_params = {}
le.get_loss_fn(le.hparams["optimizer"], {})

optim_params = {
    'params': le.model.parameters(),
    'weight_decay': le.hparams["weight_decay"],
    'lr': le.hparams["lr"]
}

le.get_optimizer(le.hparams["loss_fn"], optim_params)

# train
le.train(train_loader, val_loader,
              batch_size=le.hparams["batch_size"], n_epochs=le.hparams["n_epoch"],
              n_features=1)

'8c83641bbc8948f88acc2793f9e22e1a'

## Train breakdown

In [37]:
batch_size=64
n_epochs=10
n_features=1
import numpy as np
import torch
from sklearn.metrics import accuracy_score

In [33]:
le._logger.info("[Start] Training. ID={0}".format(le.id))
# _ = le.get_model_save_path()  #todo : designate path

# setup
le.prediction = {}
le.predictions_out = {}
le.truths = {}
le.truths_out = {}
le.out_class = eval(le.model_config.get("OUT_CLASS"))

# mlflow
dict_config = {
    "batch_size": batch_size,
    "n_epochs": n_epochs,
    "optimizer": le.optimizer_name,
    "loss_fn": le.loss_fn_name
}

le.mlwriter.create_experiment(le.id, le.mlflow_tags)
le.mlwriter.log_params_from_omegaconf_dict(dict_config)

# start
for epoch in range(1, n_epochs + 1):
    batch_losses = []
    le.train_loader = train_loader
    for x_batchs, y_batchs in train_loader:
        le.optimizer.zero_grad()
        loss = le.train_step(x_batchs, y_batchs)
        batch_losses.append(loss)
    training_loss = np.mean(batch_losses)
    le.train_losses.append(training_loss)
    le.mlwriter.log_metric('train_loss', training_loss, epoch)

    with torch.no_grad():
        batch_val_losses = []
        predictions = []
        truths = []
        for x_vals, y_val in val_loader:
            prediction, truth, val_loss = le.eval_step(x_vals, y_val)
            predictions.append(prediction)
            truths.append(truth)
            batch_val_losses.append(val_loss)

        validation_loss = np.mean(batch_val_losses)
        le.val_losses.append(validation_loss)
        
        predictions = np.concatenate(predictions)
        truths = np.concatenate(truths)

        # record
        le.prediction["val"], le.truths["val"] = predictions, truths
        predictions_out, truths_out = le.convert_model_value(predictions), le.convert_model_value(truths)
        le.predictions_out["val"], le.truths_out["val"] = predictions_out, truths_out
        acc = accuracy_score(truths_out, predictions_out)

        le.mlwriter.log_metric('valid_acc', acc, epoch)
        le.mlwriter.log_metric('valid_loss', validation_loss, epoch)
        # print(le.prediction["val"])
        # print(le.predictions_out["val"])

    if (epoch <= 10) | (epoch % 50 == 0):
        le._logger.info(
            f"[{epoch}/{n_epochs}] Training loss: {training_loss:.4f}\t Validation loss: {validation_loss:.4f}"
        )

# record last 
le.save_numpy_obj(predictions, "val_preds.csv")
le.save_numpy_obj(truths, "val_truth.csv")
le.save_numpy_obj(predictions_out, "val_preds_out.csv", type='int')
le.save_numpy_obj(truths_out, "val_truth_out.csv", type='int')

# le.save_model()
le.save_mlflow_model()
le._logger.info("[DONE] Training. ID={0}".format(le.id))


TypeError: forward() missing 2 required positional arguments: 'node_feature' and 'adj'

In [12]:
om.deploy_best_model()

Log path = /root/src/aleister/log/logging.log
